# 1. Get raw dataset from database

In [1]:
import pandas as pd
import psycopg2

def get_connection_parameters():
    print("Input the data for connection to database:")
    user = input("username >> ")
    password = input("password >> ")
    host = input("host >> ")
    port = input("port >> ")
    database = input("database >> ")
    return user, password, host, port, database

query_path = 'data_preprocessing/final_dataset_query.sql'

In [ ]:
user, password, host, port, database = get_connection_parameters()

In [1]:
def load_data_from_db(user, password, host, port, database, file_path):
    with open(file_path, 'r') as file:
        sql_query = file.read()

    conn = psycopg2.connect(
            user=user,
            password=password,
            host=host,
            port=port,
            database=database
        )
    
    try:
        df = pd.read_sql_query(sql_query, conn)
    finally:
        conn.close()
    
    return df

In [ ]:
df = load_data_from_db(user, password, host, port, database, query_path)

# 2. Preprocessing of data

In [ ]:
# remove all consecutive spaces and line breaks
df['full_content'] = df['full_content'].replace(r'\n', '', regex=True).replace(r'\s+', ' ', regex=True).str.lower()

# 3. Extracting features (items)

## 3.1. Extracting Management discussion and analysis

In [ ]:
def extract_matching_text(text, pattern):
    matches = re.findall(pattern, text, re.IGNORECASE)
    if matches:
        return ' '.join(matches)
    return None 

pattern_mda = r'ITEM.{,20}MANAGEMENT.{,10}DISCUSSION.{,10}ANALYSIS.{,10}OF.{,10}FINANCIAL(.*?)ITEM.{0,3}\d.{0,4}QUANTITATIVE'

df['MDA'] = df['full_content'].apply(lambda x: extract_matching_text(x, pattern_mda))

## 3.2. Extracting Quantitative and Qualitative Disclosures About Market Risk

In [ ]:
pattern_market = r'ITEM.{,20}QUANTITATIVE.{,10}AND.{,10}QUALITATIVE.{,10}DISCLOSURES.{,10}ABOUT(.*?)ITEM.{0,3}\d.{0,4}CONTROLS'

df['MARKET_RISK'] = df['full_content'].apply(lambda x: extract_matching_text(x, pattern_market))

# 3.3. Extracting Risk Factors

pattern_risks = r'ITEM.{,20}RISK.{,10}FACTORS.{,10}(.*?)ITEM.{0,3}\d.{0,4}UNREGISTERED'

df['RISK_FACTORS'] = df['full_content'].apply(lambda x: extract_matching_text(x, pattern_risks))